In [3]:
import wikipediaapi

def get_out_degree(title):
    # Set a compliant user agent
    user_agent = 'MyWikiApp/1.0 (contact@example.com)'  # Replace with real contact
    wiki = wikipediaapi.Wikipedia(user_agent=user_agent, language='en')

    page = wiki.page(title)

    if not page.exists():
        print(f"❌ Page '{title}' does not exist.")
        return 0

    out_links = page.links.keys()
    print(f"Page: {title}")
    print(f"Out-degree (number of outbound links): {len(out_links)}")
    return len(out_links)

# Example usage:
get_out_degree("United States")

Page: United States
Out-degree (number of outbound links): 1997


1997

In [1]:
print("Hello, World!")

Hello, World!
